In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/clase_11/clase_11/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Pandas 2

## Introducción


En esta notebook vamos a trabajar con los conceptos de agregación a través de `groupby` y `pivot tables`. 

Analizaremos características de precio y superficie, por barrio y por comuna, de inmuebles en venta en la Ciudad de Buenos Aires en el año 2016.


## Dataset

El Gobierno de la Ciudad de Buenos Aires disponibiliza algunos datasets para acceso público en la url 
https://data.buenosaires.gob.ar/

Allí encontramos datos de propiedades inmuebles en venta, separados por año
https://data.buenosaires.gob.ar/dataset/departamentos-venta

En esta clase vamos a usar un subconjunto de los datos de "Departamentos en venta 2016" que descargamos desde aquí
https://data.buenosaires.gob.ar/dataset/departamentos-venta/archivo/juqdkmgo-7031-resource

En la carpeta Data de esta clase ya tenemos descargardo ese dataset, el nombre del archivo es **departamentos-en-venta-2016.csv**.

## Imports

In [2]:
import pandas as pd
import numpy as np

## Ejercicio 1  - Importar 

Leamos los datos del archivo departamentos-en-venta-2016.csv

Veamos cuántos registros tiene y de qué tipos son los datos de cada columna. 

Veamos los primeros registros para verificar que los datos fueron importados correctamente.

In [3]:
import chardet
def get_encoding_type(csv_path):
    rawdata = open(csv_path, 'rb').read()
    result = chardet.detect(rawdata)
    return result.get('encoding')

In [4]:
data_location = "../Data/departamentos-en-venta-2016.csv"

In [8]:
get_encoding_type(data_location)

'utf-8'

In [5]:
data = pd.read_csv(data_location, sep = ';')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7564 entries, 0 to 7563
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CALLE                    6182 non-null   object 
 1   NUMERO                   6182 non-null   float64
 2   ID_ZONAPRO               7564 non-null   int64  
 3   OPERACION                7564 non-null   object 
 4   TIPO                     7564 non-null   object 
 5   M2                       7564 non-null   int64  
 6   M2CUB                    7564 non-null   int64  
 7   PRECIOTEXT               7562 non-null   object 
 8   PRECIOARS                7564 non-null   int64  
 9   PRECIOARSM               7564 non-null   int64  
 10  DOLARES                  7564 non-null   int64  
 11  U_S_M2                   7564 non-null   int64  
 12  AMBIENTES                7564 non-null   int64  
 13  ANTIGUEDAD               7564 non-null   int64  
 14  BAÑOS                   

In [7]:
data.head(3)

,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,URL,REVISION,NOTA,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD
0,GUATEMALA,5574.0,42408691,VTA,DTO,57,50,U$S 170.150,2977625,59553,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42408...,NaN,NaN,GUATEMALA 5574,PALERMO,COMUNA 14,1425.0,C1425BVH,-34.580581,-58.431758
1,ZAPATA,300.0,42408710,VTA,DTO,46,46,U$S 118.650,2076375,45139,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42408...,NaN,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609
2,ZAPATA,300.0,42518390,VTA,DTO,61,56,U$S 181.470,3175725,56709,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42518...,NaN,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609


## Ejercicio 2  - Promedio

¿Cuál es el promedio valor del precio por $m^2$ en dolares?

Para responder esta pregunta vamos a:

1) Crear un obeto Series que resulte de limpiar los valores del campo PRECIOTEXT. Tenemos que eliminar los símbolos "U\\$S" y "."

2) En la instancia de Series resultado del punto 1) reemplazamos los valores en pesos ('\\$') por nulos (para que nos ensucien los datos de promedio). 

3) Crear una columna nueva de tipo numérico en el DataFrame (PRECIOTEXT_CLEAN) y asignar los valores de resultado de 2)

3) Crear una columna de tipo numérico (PRECIOTEXTM) que tenga el precio del m2 en dolares calculado como el valor de PRECIOTEXT_CLEAN / M2

4) Calcular la media de los valores de PRECIOTEXTM


In [9]:
df = data.copy()

In [18]:
df.drop(['REVISION','NOTA'], inplace=True, axis=1)

In [20]:
df.head()

,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,PUBLICADO,PROCESADO,URL,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD
0,GUATEMALA,5574.0,42408691,VTA,DTO,57,50,U$S 170.150,2977625,59553,...,27/11/2016,12/10/2017,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42408...,GUATEMALA 5574,PALERMO,COMUNA 14,1425.0,C1425BVH,-34.580581,-58.431758
1,ZAPATA,300.0,42408710,VTA,DTO,46,46,U$S 118.650,2076375,45139,...,27/11/2016,12/10/2017,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42408...,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609
2,ZAPATA,300.0,42518390,VTA,DTO,61,56,U$S 181.470,3175725,56709,...,27/11/2016,11/10/2017,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42518...,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609
3,ZAPATA,300.0,42518402,VTA,DTO,140,76,U$S 320.000,5600000,73684,...,27/11/2016,11/10/2017,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42518...,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609
4,"JUSTO, JUAN B. AV.",2300.0,42621693,VTA,DTO,39,33,U$S 82.116,1437030,43546,...,27/7/2016,10/10/2017,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42621...,"JUSTO, JUAN B. AV. 2300",VILLA CRESPO,COMUNA 15,1414.0,C1414CWY,-34.592984,-58.441542


In [44]:
import re

patron_usd = re.compile(r'U\$S')
patron_dot = re.compile(r'\.')

sin_usd = df["PRECIOTEXT"].apply(lambda x: x if x is np.NaN else re.sub(patron_usd,'',str(x)).strip())
sin_usd_dot = sin_usd.apply(lambda x: x if x is np.NaN else re.sub(patron_dot,'',str(x)).strip())

patron_peso = re.compile(r'\$')

en_pesos = sin_usd_dot.apply(lambda pesos: pesos if pesos is np.NaN else patron_peso.match(str(pesos)))

pesos_mask = en_pesos.notnull()

sin_usd_dot.loc[pesos_mask]=np.NaN

df['PRECIOTEXT_CLEAN']=sin_usd_dot.astype(float)

In [42]:
df.columns

Index(['CALLE', 'NUMERO', 'ID_ZONAPRO', 'OPERACION', 'TIPO', 'M2', 'M2CUB',
       'PRECIOTEXT', 'PRECIOARS', 'PRECIOARSM', 'DOLARES', 'U_S_M2',
       'AMBIENTES', 'ANTIGUEDAD', 'BAÑOS', 'DIRECCION', 'LOCATION',
       'PUBLICADO', 'PROCESADO', 'URL', 'DIRECCION_NORMALIZADA', 'BARRIO',
       'COMUNA', 'CODIGO_POSTAL', 'CODIGO_POSTAL_ARGENTINO', 'LATITUD',
       'LONGITUD', 'PRECIOTEXT_CLEAN'],
      dtype='object')

In [45]:
df["M2PRICE"] = df[['PRECIOTEXT_CLEAN','M2']].apply(lambda x: x['PRECIOTEXT_CLEAN']/x['M2'], axis=1)

In [49]:
media_preciom2=round(df["M2PRICE"].mean(),2)
media_preciom2

2522.49

## Ejercicio 3  - Promedio por cuartil
¿Cuál es el precio promedio del metro cuadrado en dolares para cada cuartil de superficie (campo M2) de las viviendas en CABA?

Comenzamos calculando los cuartilos de superficie.

In [50]:
cuartiles = pd.qcut(df.M2, 4)
cuartiles

0         (54.0, 80.0]
1         (41.0, 54.0]
2         (54.0, 80.0]
3        (80.0, 730.0]
4       (14.999, 41.0]
             ...      
7559    (14.999, 41.0]
7560      (41.0, 54.0]
7561      (41.0, 54.0]
7562      (41.0, 54.0]
7563      (41.0, 54.0]
Name: M2, Length: 7564, dtype: category
Categories (4, interval[float64]): [(14.999, 41.0] < (41.0, 54.0] < (54.0, 80.0] < (80.0, 730.0]]

In [52]:
df.groupby(cuartiles)['M2PRICE'].mean()

M2
(14.999, 41.0]    2418.485356
(41.0, 54.0]      2455.988147
(54.0, 80.0]      2438.613868
(80.0, 730.0]     2788.299117
Name: M2PRICE, dtype: float64

## Ejercicio 3  - Promedio por barrio

¿Cuál es la media de precio por metro cuadrado en dolares para cada barrio de la CABA? 

Ordenemos los datos para indicar cuál es el barrio más caro.

Resolvamos el cálculo tanto con groupby como con pivot tables

In [55]:
df.groupby('BARRIO')["M2PRICE"].mean().sort_values(ascending= False)[:5]

BARRIO
PUERTO MADERO    5412.468738
PALERMO          3136.014984
BELGRANO         3110.065913
RETIRO           2941.052687
RECOLETA         2876.201544
Name: M2PRICE, dtype: float64

In [59]:
b=pd.pivot_table(df, index='BARRIO', aggfunc={'M2PRICE':'mean'})

b.sort_values(['M2PRICE'], ascending = False)[:5]

,M2PRICE
BARRIO,
PUERTO MADERO,5412.468738
PALERMO,3136.014984
BELGRANO,3110.065913
RETIRO,2941.052687
RECOLETA,2876.201544


## Ejercicio 4  - Dispersión de precios por barrio

**4.a** ¿En qué barrio hay una mayor dispersión en el valor del metro cuadrado en dolares? Ordenar los valores para identificar el mayor.

Ayuda: Calculamos dispersión como el desvío estandar de un grupo dividido la media de ese grupo grupo

Vamos a calcular esto de dos maneras distintas:

1) Calculamos la instancia de Series que tiene la media por grupo. Calculamos la instancia de Series que tiene el desvío estandar por grupo. Las dividimos. Ordenamos

2) Calculamos los grupos y usamos una función lambda que calcule la media, desvío estandar y divida. Ordenamos.

**4.b** Cuál les parece que será más eficiente? Probemoslo con %timeit, para eso definamos dos funciones que encapsulen el código de 1) y 2).

**4.c** ¿Por qué Paternal está devolviendo NaN? ¿Y por qué Villa Soldati devuelve 0?

Nota: tengan en cuenta que `mean` y `std` sobre una instancia de `DataFrameGroupBy` excluye los valores nulos

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.mean.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.std.html





dispersion = std()/mean()    

In [70]:
desvios = df.groupby("BARRIO")['M2PRICE'].std()
means = df.groupby('BARRIO')['M2PRICE'].mean()
dispersion = desvios / means
dispersion.sort_values(ascending = False)

BARRIO
VILLA DEVOTO         0.440995
BOCA                 0.425274
MONSERRAT            0.321376
PALERMO              0.308486
BALVANERA            0.300993
NUÑEZ                0.295307
VELEZ SARSFIELD      0.275630
RECOLETA             0.263813
CHACARITA            0.249468
SAN TELMO            0.248734
FLORES               0.248200
BELGRANO             0.245323
RETIRO               0.244298
PUERTO MADERO        0.234363
MATADEROS            0.229643
VILLA LURO           0.227212
COLEGIALES           0.217556
BARRACAS             0.212347
SAAVEDRA             0.212152
CONSTITUCION         0.208357
LINIERS              0.198838
ALMAGRO              0.191560
MONTE CASTRO         0.188504
VILLA URQUIZA        0.185133
VILLA CRESPO         0.182353
VILLA DEL PARQUE     0.180029
SAN CRISTOBAL        0.176112
SAN NICOLAS          0.170378
VILLA GRAL. MITRE    0.168801
CABALLITO            0.167364
COGHLAN              0.164022
BOEDO                0.150507
VILLA PUEYRREDON     0.149338
PAR

Vamos a ver cuánto da si usamos la lambda directamente, sin pasar por dispersion_2:

In [ ]:
Calculamos los grupos y usamos una función lambda que calcule la media, desvío estandar y divida. Ordenamos

In [82]:
dispersion_2 = df.groupby('BARRIO').apply(lambda x: x['M2PRICE'].std() / x['M2PRICE'].mean())
dispersion_2.sort_values(ascending =False)

BARRIO
VILLA DEVOTO         0.440995
BOCA                 0.425274
MONSERRAT            0.321376
PALERMO              0.308486
BALVANERA            0.300993
NUÑEZ                0.295307
VELEZ SARSFIELD      0.275630
RECOLETA             0.263813
CHACARITA            0.249468
SAN TELMO            0.248734
FLORES               0.248200
BELGRANO             0.245323
RETIRO               0.244298
PUERTO MADERO        0.234363
MATADEROS            0.229643
VILLA LURO           0.227212
COLEGIALES           0.217556
BARRACAS             0.212347
SAAVEDRA             0.212152
CONSTITUCION         0.208357
LINIERS              0.198838
ALMAGRO              0.191560
MONTE CASTRO         0.188504
VILLA URQUIZA        0.185133
VILLA CRESPO         0.182353
VILLA DEL PARQUE     0.180029
SAN CRISTOBAL        0.176112
SAN NICOLAS          0.170378
VILLA GRAL. MITRE    0.168801
CABALLITO            0.167364
COGHLAN              0.164022
BOEDO                0.150507
VILLA PUEYRREDON     0.149338
PAR

Veamos por qué Paternal devuelve NaN y Villa Soldati que devuelve 0.


In [87]:
paternal=df.BARRIO == 'PATERNAL'
df.loc[paternal,'M2PRICE']

3976    1720.779221
Name: M2PRICE, dtype: float64

In [ ]:
Me da NaN en paternal, porque tengo 1 solo registro, entonces no puede calcular el desvio std, por lo tanto, me tira NaN Value

In [93]:
soldati=df.BARRIO == 'VILLA SOLDATI'
df.loc[soldati,'M2PRICE']

3059    812.5
3060    812.5
Name: M2PRICE, dtype: float64

In [ ]:
Me da 0, ya que solo tengo 2 registros con el mismo valor, por lo tanto su desviacion std es 0

## Ejercicio 5  - Superficie por barrio

Calculemos la mediana de superficie por barrio para determinar en qué barrio los departamentos son más grandes.

Hagamos el cálculo con pivot_table y groupby


In [99]:
tamanio = df.groupby('BARRIO')['M2'].median()
tamanio[:5]

BARRIO
AGRONOMIA    40.0
ALMAGRO      53.0
BALVANERA    55.0
BARRACAS     65.5
BELGRANO     74.0
Name: M2, dtype: float64

In [100]:
tamanio2=pd.pivot_table(df, index = 'BARRIO' , aggfunc = {'M2':'median'})
tamanio2

,M2
BARRIO,
AGRONOMIA,40.0
ALMAGRO,53.0
BALVANERA,55.0
BARRACAS,65.5
BELGRANO,74.0
BOCA,59.5
BOEDO,47.0
CABALLITO,54.0
CHACARITA,45.0


## Ejercicio 6  - Tabla resumen

Generemos un `DataFrame` que agregue la información del precio por M2 en dolares (`PRECIOTEXTM`), ambientes (`AMBIENTES`) a nivel de `COMUNA` y barrio (`BARRIO`). Proporcione información tanto de la tendencia central como de la dispersión de ambas distribuciones.

Ayuda: usar pivot_table

In [113]:
df.pivot_table(['M2PRICE','AMBIENTES'], index = ["COMUNA","BARRIO"], 
                                       aggfunc = {'M2PRICE':[np.mean, np.std], 'AMBIENTES': [np.mean, np.std]})

AMBIENTES                M2PRICE             
                                 mean       std         mean          std
COMUNA    BARRIO                                                         
COMUNA 01 CONSTITUCION       2.208696  0.893403  2197.241584   457.810983
          MONSERRAT          2.100000  1.087115  2349.127916   754.954231
          PUERTO MADERO      2.678571  0.765320  5412.468738  1268.483199
          RETIRO             2.892473  1.684084  2941.052687   718.493943
          SAN NICOLAS        2.190698  1.232696  2186.049883   372.454324
          SAN TELMO          2.256410  0.594623  2145.817227   533.737353
COMUNA 02 RECOLETA           2.931204  1.353770  2876.201544   758.780248
COMUNA 03 BALVANERA          2.476636  1.046755  1988.284935   598.460600
          SAN CRISTOBAL      2.397869  0.621211  1914.203956   337.113408
COMUNA 04 BARRACAS           2.583333  1.088918  2166.146707   459.975468
          BOCA               2.312500  1.138347  1963.575157   835.057816
          NUEVA POMPEYA      2.000000  0.000000  1622.727273   109.280139
          PARQUE PATRICIOS   2.208955  0.640485  1943.392373   223.179437
COMUNA 05 ALMAGRO            2.370149  1.094490  2252.269793   431.445048
          BOEDO              2.032258  1.224306  2215.391462   333.432291
COMUNA 06 CABALLITO          2.528771  0.947383  2412.777566   403.811538
COMUNA 07 FLORES             2.806452  0.875488  1905.204204   472.870879
          PARQUE CHACABUCO   2.330579  0.541969  2645.412539   329.268919
COMUNA 08 VILLA LUGANO       2.333333  1.211060  1138.726815   153.217663
          VILLA SOLDATI      4.000000  0.000000   812.500000     0.000000
COMUNA 09 LINIERS            2.290323  0.662511  1885.041244   374.817469
          MATADEROS          2.727273  0.674200  1804.158874   414.312201
          PARQUE AVELLANEDA  2.000000  0.000000  1806.324289   108.443814
COMUNA 10 FLORESTA           2.277778  1.017815  1711.078346   210.982377
          MONTE CASTRO       2.400000  0.709721  1952.634631   368.079223
          VELEZ SARSFIELD    3.200000  2.049390  1748.636059   481.976635
          VERSALLES          3.000000  0.000000  1926.168413   166.805407
          VILLA LURO         2.705882  1.046704  1960.143435   445.368473
          VILLA REAL         2.666667  0.577350  1738.200049    62.479211
COMUNA 11 VILLA DEL PARQUE   2.692308  1.319674  2240.717867   403.395202
          VILLA DEVOTO       2.653846  1.809802  2190.415302   965.962914
          VILLA GRAL. MITRE  2.285714  0.673024  1928.578367   325.546393
          VILLA SANTA RITA   2.137931  0.789400  1907.998258   201.952963
COMUNA 12 COGHLAN            2.684211  0.808912  2451.120924   402.036953
          SAAVEDRA           2.204819  1.000110  2365.061414   501.751822
          VILLA PUEYRREDON   2.000000  0.946864  2159.102203   322.435274
          VILLA URQUIZA      2.157609  1.009326  2402.505658   444.782207
COMUNA 13 BELGRANO           2.767442  1.125982  3110.065913   762.970942
          COLEGIALES         2.726316  1.046190  2636.723717   573.635491
          NUÑEZ              2.595745  0.999848  2597.472491   767.050772
COMUNA 14 PALERMO            2.491256  1.247419  3136.014984   967.417254
COMUNA 15 AGRONOMIA          1.833333  0.983192  2276.703779   192.974291
          CHACARITA          1.764706  1.087901  2338.939470   583.490399
          PARQUE CHAS        1.937500  0.573730  2137.475822   290.456966
          PATERNAL           3.000000       NaN  1720.779221          NaN
          VILLA CRESPO       2.255639  0.838384  2331.951318   425.238932
          VILLA ORTUZAR      2.264151  0.593262  2408.765643   282.605371